In [15]:
# nltk module
import nltk
from nltk.stem.lancaster import LancasterStemmer

# module we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random
import json


In [16]:
# create a stemmer
stemmer = LancasterStemmer()

# load intents
# import our chat-bot intents file
import json
with open('intents_bn.json') as json_data:
    intents = json.load(json_data)




In [17]:
words = []
classes = []
documents = []
ignore_words = []


for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word into a sentence
        w = nltk.word_tokenize(pattern)
        # add word to our words list
        words.extend(w)
        # add documents to our corpus
        documents.append((w, intent['tag']))

        # add to our classses
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


# stem and lower case the word and remove duplicates
words =  [stemmer.stem(w.lower()) for w in words ]




# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))

In [18]:

# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

12 documents
3 classes ['অভিবাদন', 'ধন্যবাদ', 'বিদায়']
26 unique stemmed words [',', '?', 'অান্তরিক', 'আছেন', 'আপনাকে', 'আপনি', 'আমাকে', 'করার', 'কেউ', 'কেমন', 'চ্যাট', 'জন্য', 'দিন', 'দেখা', 'ধন্যবাদ', 'পরবর্তী', 'পরে', 'পর্যন্ত', 'বিদায়', 'ভাল', 'শুভ', 'সময়', 'সাহায্য', 'হবে', 'হাই', 'হ্যালো']


In [19]:
# # create our training data
# training = []
# # create an empty array for our output
# output_empty = [0] * len(classes)
# for doc in documents:
#     bag = []
#     pattern_words = doc[0]
#     pattern_words = [stemmer.stem(word.lower()) for word in pattern_words ]
    
#     for w in words:
#         bag.append(1) if w in pattern_words else bag.append(0)
        
#     output_row = list(output_empty)
#     output_row[classes.index(doc[1])] = 1
#     traininga.append(bag, output_row)


# output_row

In [20]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

In [21]:
training[0]

array([list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
       list([1, 0, 0])], dtype=object)

In [22]:

# create train and test lists. X - patterns, Y - intents

train_x = list(training[:,0])

train_y = list(training[:,1])

print(train_x[:1], train_y[:1])
len(train_x), len(train_y)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]] [[1, 0, 0]]


(12, 12)

In [23]:

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons

# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [24]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 128)               3456      
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 195       
Total params: 11,907
Trainable params: 11,907
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Fit the model
history = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
12/12 [==============================] - 0s 6ms/step - loss: 1.1563 - accuracy: 0.3333
Epoch 2/200
12/12 [==============================] - 0s 285us/step - loss: 1.1845 - accuracy: 0.2500
Epoch 3/200
12/12 [==============================] - 0s 268us/step - loss: 1.0796 - accuracy: 0.5000
Epoch 4/200
12/12 [==============================] - 0s 419us/step - loss: 1.0502 - accuracy: 0.5000
Epoch 5/200
12/12 [==============================] - 0s 372us/step - loss: 1.0201 - accuracy: 0.4167
Epoch 6/200
12/12 [==============================] - 0s 712us/step - loss: 0.9355 - accuracy: 0.5833
Epoch 7/200
12/12 [==============================] - 0s 1ms/step - loss: 0.8634 - accuracy: 0.7500
Epoch 8/200
12/12 [==============================] - 0s 591us/step - loss: 0.9418 - accuracy: 0.7500
Epoch 9/200
12/12 [==============================] - 0s 276us/step - loss: 0.9030 - accuracy: 0.7500
Epoch 10/200
12/12 [==============================] - 0s 668us/step - loss: 0.7307 - accuracy: 

12/12 [==============================] - 0s 967us/step - loss: 0.0056 - accuracy: 1.0000
Epoch 82/200
12/12 [==============================] - 0s 496us/step - loss: 0.0180 - accuracy: 1.0000
Epoch 83/200
12/12 [==============================] - 0s 703us/step - loss: 0.0963 - accuracy: 1.0000
Epoch 84/200
12/12 [==============================] - 0s 936us/step - loss: 0.0571 - accuracy: 1.0000
Epoch 85/200
12/12 [==============================] - 0s 462us/step - loss: 0.0284 - accuracy: 1.0000
Epoch 86/200
12/12 [==============================] - 0s 750us/step - loss: 0.0975 - accuracy: 0.9167
Epoch 87/200
12/12 [==============================] - 0s 397us/step - loss: 0.0197 - accuracy: 1.0000
Epoch 88/200
12/12 [==============================] - 0s 640us/step - loss: 0.0488 - accuracy: 1.0000
Epoch 89/200
12/12 [==============================] - 0s 656us/step - loss: 0.0392 - accuracy: 1.0000
Epoch 90/200
12/12 [==============================] - 0s 400us/step - loss: 0.0361 - accuracy: 

12/12 [==============================] - 0s 468us/step - loss: 0.0085 - accuracy: 1.0000
Epoch 161/200
12/12 [==============================] - 0s 744us/step - loss: 0.0040 - accuracy: 1.0000
Epoch 162/200
12/12 [==============================] - 0s 776us/step - loss: 1.4105e-04 - accuracy: 1.0000
Epoch 163/200
12/12 [==============================] - 0s 439us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 164/200
12/12 [==============================] - 0s 758us/step - loss: 0.0061 - accuracy: 1.0000
Epoch 165/200
12/12 [==============================] - 0s 586us/step - loss: 0.0417 - accuracy: 1.0000
Epoch 166/200
12/12 [==============================] - 0s 391us/step - loss: 6.8648e-04 - accuracy: 1.0000
Epoch 167/200
12/12 [==============================] - 0s 661us/step - loss: 0.0456 - accuracy: 1.0000
Epoch 168/200
12/12 [==============================] - 0s 404us/step - loss: 0.0025 - accuracy: 1.0000
Epoch 169/200
12/12 [==============================] - 0s 617us/step - loss: 0.

In [26]:
# save model to file
pickle.dump(model, open("./pickle/ahsbot-model_bn.pkl", "wb"))

# save all of our data structures
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "./pickle/ahsbot-data_bn.pkl", "wb" ) )